# **Modelling and Evaluation Notebook**

## Objectives
Answer Business requirement 2: Binary Classification using Convolutional Neural Networks

* predict if a given leaf is infected or not judging by the presence of powdery mildew.
* use the CNN to map relationships between features and labels.
* build a binary classifier and generate reports.

## Inputs

* inputs/cherry-leaves-dataset/cherry-leaves/train
* inputs/cherry-leaves-dataset/cherry-leaves/test
* inputs/cherry-leaves-dataset/cherry-leaves/validation
* image_shape.pkl (image shape embeddings pickle file)

## Outputs

* cherry-tree-model.h5 (model)
* class_distribution.png
* class_indices.pkl
* model_training_acc.png
* model_training_losses.png
* test-evaluation.pkl
* train-evaluation.pkl
* val-evaluation.pkl




---

### Define model version
* v6 is current per Nov 2024

In [2]:
version = 'v6'

### Import regular packages

In [3]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from matplotlib.image import imread

### Change working directory

Change the working directory from current folder to parent folder for root access

In [ ]:
import os
current_dir = os.getcwd()
current_dir

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

### Set input directory

In [7]:
my_data_dir = 'inputs/cherry-leaves-dataset/cherry-leaves'
train_path = my_data_dir + '/train'
val_path = my_data_dir + '/validation'
test_path = my_data_dir + '/test'

### Set output directory

In [ ]:

file_path = f'outputs/{version}'

if 'outputs' in os.listdir(current_dir) and version in os.listdir(current_dir + '/outputs'):
    print('Old version is already available, create a new version.')
    pass
else:
    os.makedirs(name=file_path)

### Gather labels

In [ ]:
try:
    labels = os.listdir(train_path)
except:
    labels = ['healthy', 'powdery_mildew']

print(f"Project Labels: {labels}")

### Load image shape embeddings

In [ ]:
import os
import joblib

try:
    image_shape = joblib.load(filename=f"outputs/{version}/image_shape.pkl")

except:
    print(
    'An error has occurred whilst importing image_shape.pkl\n'
    'Please ensure file exists and accessed via root directory.\n'
    f'You are currently in: {os.getcwd()}'
    )
    # google collab: update as needed
    print('Hardcoding provided image shape of (256, 256, 3)')
    image_shape = (256, 256, 3)
    print(f'image_shape is: {image_shape}')

else:
    print(f'image_shape is: {image_shape}')




***

# Review class distribution

* across whole dataset
* per train, test, and validation

In [ ]:
df_freq = pd.DataFrame([])
total_images_count = 0

# gather info
for folder in ['train', 'validation', 'test']:
    for label in labels:

        path = my_data_dir + '/' + folder + '/' + label

        image_count = int(len(os.listdir(path)))
        total_images_count += image_count

        try:
            df_freq = df_freq.append(pd.Series({
                'Set': folder.capitalize(), 'Label': label, 'Frequency': image_count}), ignore_index=True)
            print(f"* {folder} - {label}: {image_count} images\n")
        except:
            # for google collab functionality
            df_freq = df_freq.concat(pd.Series({
                'Set': folder.capitalize(), 'Label': label, 'Frequency': image_count}), ignore_index=True)
            print(f"* {folder} - {label}: {image_count} images\n")


print(f'{total_images_count} images total')
print('--------')

# plot class distribution
plt.figure(figsize=(10, 6))
sns.set_style('darkgrid', {'grid.color': '.6', 'grid.linestyle': '(0.5, 0.5)', 'xtick.bottom' : True })

sns.barplot(
    data=df_freq, x='Set', y='Frequency',
    hue='Label', palette='flare', linewidth=0.5
    )



plt.title('Class Distribution')
plt.savefig(f'{file_path}/class_distribution.png',
            bbox_inches='tight', dpi=600)
plt.show()
print('\n')

print('--------')

# confirm percentages of dataset
df_freq.set_index('Label', inplace=True)
df_freq['Percent of DataSet'] = round(
    df_freq['Frequency'] / total_images_count * 100)

print(df_freq)

We can confirm that train, validation and test set percentages of dataset are split as expected, and that there are equal amounts of both classes (healthy and powdery_mildew) in each set.

***

# Image Augmentation

### Define image data generator, initialize


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Initialize
augmented_image_data = ImageDataGenerator(rotation_range=20,
                                          width_shift_range=0.1,
                                          height_shift_range=0.1,
                                          shear_range=0.1,
                                          zoom_range=0.1,
                                          horizontal_flip=True,
                                          vertical_flip=True,
                                          fill_mode='nearest',
                                          rescale=1./255
                                          )

### Define batch size 

In [12]:
batch_size = 20

### Augment TRAINING image dataset


In [ ]:
train_set = augmented_image_data.flow_from_directory(train_path,
                                                     target_size=image_shape[:2],
                                                     color_mode='rgb',
                                                     batch_size=batch_size,
                                                     class_mode='binary',
                                                     shuffle=True,
                                                     seed=42
                                                     )


train_set.class_indices


train_set.class_indices

### Rescale validation image dataset


In [ ]:
validation_set = ImageDataGenerator(rescale=1./255).flow_from_directory(val_path,
                                                          target_size=image_shape[:2],
                                                          color_mode='rgb',
                                                          batch_size=batch_size,
                                                          class_mode='binary',
                                                          shuffle=False
                                                          )

validation_set.class_indices

### Rescale test image dataset

In [ ]:
test_set = ImageDataGenerator(rescale=1./255).flow_from_directory(test_path,
                                                    target_size=image_shape[:2],
                                                    color_mode='rgb',
                                                    batch_size=batch_size,
                                                    class_mode='binary',
                                                    shuffle=False
                                                    )

test_set.class_indices

### Plot augmented training images

In [ ]:
for _ in range(3):
    try:
        img, label = train_set.next()
    except:
        # for google collab functionality
        img, label = next(train_set)

    print(f'{img.shape}\n')  # expect: (20, 256, 256, 3)
    plt.imshow(img[0])
    print('--------------')
    plt.show()


### Plot augmented validation and test images

In [ ]:
# validation_set
for _ in range(3):
    try:
        img, label = validation_set.next()
    except:
        # for google collab functionality
        img, label = next(validation_set)
    print(f'{img.shape}\n')
    plt.imshow(img[0])
    print('--------------')
    plt.show()



In [ ]:
# test set
for _ in range(3):
    try:
        img, label = test_set.next()
    except:
        # for google collab functionality
        img, label = next(test_set)
    print(f'{img.shape}\n')
    plt.imshow(img[0])
    print('--------------')
    plt.show()

###  Observations
Augmented validation and test images have been standardized between 0 to 255 pixels. As you can see, the images are ugmented and are ready to be used for developing and training a CNN model.

### Save class indices

In [ ]:
joblib.dump(value=train_set.class_indices,
            filename=f"{file_path}/class_indices.pkl")

---

# Model Creation

---

### ML Model

* Import model packages

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger
from tensorflow.keras.utils import plot_model

* ### Model

In [21]:
def create_tf_model():
    """
    Creates a CNN model for binary classification of leaf images
    """
    model = Sequential()

    # Input layer: CONV1
    model.add(Conv2D(filters=32, kernel_size=(3, 3),
        input_shape=image_shape,  # average image shape
        activation='relu', ))
    model.add(MaxPooling2D(pool_size=(2,2)))

    # CONV2
    model.add(Conv2D(filters=12, kernel_size=(3, 3),
        activation='relu', ))
    model.add(MaxPooling2D(pool_size=(2,2)))

    # CONV3
    model.add(Conv2D(filters=8, kernel_size=(3,3),
        activation='relu', ))
    model.add(MaxPooling2D(pool_size=(2,2)))

    # Flatten
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))

    # Output
    model.add(Dense(1, activation='sigmoid'))

    # Compile
    model.compile(
        loss='binary_crossentropy',
        optimizer='adam',
        metrics=['accuracy',])

    return model

## Model summary

In [ ]:
summary = create_tf_model().summary()

Plot model

In [ ]:
try:
    plot_model(model, show_shapes=True, to_file=f'model_{version}.png')
except Exception as e:
    print(e)
    pass


Add early stopping
* Avoid overfitting

In [25]:
early_stop = EarlyStopping(monitor='val_loss', patience=3, verbose=3)

In [ ]:
# TODO remove before submit?
print(f'Train set object: {train_set}')
print(f"Number of samples in training set: {train_set.samples}")
print(f"Number of classes: {len(train_set.classes)}")
print(f"Batch size: {batch_size}")
print(f"Current steps calculation (classes/batch_size): {len(train_set.classes) // batch_size}")
print(f"Correct steps calculation (samples/batch_size): {train_set.samples // batch_size}")
print(f"Number of validation samples: {validation_set.samples}")
print(f"Validation steps per epoch: {validation_set.samples // batch_size}")

***

# Fit Model for training

### Save checkpoints

In [28]:
import os

# check if the full path exists
checkpoint_folder = f'/workspace/outputs/{version}/training_checkpoints'

if not os.path.exists(checkpoint_folder):
    os.makedirs(checkpoint_folder)
    print('training checkpoints folder made')

# dynamically include the epoch in checkpoint file name
checkpoint_path = f"{checkpoint_folder}/cp-{{epoch:04d}}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# callback to save model weights per epoch
cp_callback = ModelCheckpoint(filepath=checkpoint_path,
                                                 verbose=1,
                                                 save_weights_only=True,
                                                 save_freq='epoch',
                                                 monitor='accuracy',
                                                 save_best_only=True)


Brief untrained model evaluation

In [ ]:
loss, acc = create_tf_model().evaluate(test_set, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))  # 2 decimal places

Add CSV logger for history access in case of training runtime errors

In [31]:
history_csv_logger = CSVLogger('training.log', separator=',', append=False)

### Set epochs

In [32]:
EPOCHS = 25

In [ ]:
try:
    model = create_tf_model()
    model.fit(train_set,
            epochs=EPOCHS,
            steps_per_epoch = len(train_set.classes) // batch_size,
            validation_data=validation_set,
            callbacks=[early_stop, cp_callback, history_csv_logger],
            verbose=1,
            )
except Exception as e:
    print(e + '\n')

    # load latest weights
    latest = tf.train.latest_checkpoint(checkpoint_dir)
    model.load_weights(latest)

    print('Model restored! Continuing...')
    # continue model training
    model.fit(train_set,
            epochs=EPOCHS,
            steps_per_epoch = len(train_set.classes) // batch_size,
            validation_data=validation_set,
            callbacks=[early_stop, cp_callback, history_csv_logger],
            verbose=1,
            )


### Save model

In [ ]:
try:
    model.save(f'outputs/{version}/cherry-tree-model.h5')
    print('model saved!')
except Exception as e:
    print(e)

*** 

# Evaluate Model Performance 

### Plot model training loss and accuracy 

In [ ]:
# TODO evaluate if this or above is better or export as pickle file for interactive?
losses = pd.DataFrame(model.history.history)

sns.set_style("darkgrid")
losses[['loss','val_loss']].plot(style='.-')
plt.title("Loss")
plt.savefig(f'{file_path}/model_training_losses.png', bbox_inches='tight', dpi=150)
plt.show()


print("\n")
losses[['accuracy','val_accuracy']].plot(style='.-')
plt.title("Accuracy")
plt.savefig(f'{file_path}/model_training_acc.png', bbox_inches='tight', dpi=150)
plt.show()

### Evaluate and save

In [ ]:
from keras.models import load_model

model = load_model(f'{file_path}/cherry-tree-model.h5')

In [ ]:
try:
    eval1 = model.evaluate(test_set)
    evaluation_train = model.evaluate(train_set)
    evaluation_val = model.evaluate(validation_set)
except Exception as e:
    eval1 = model.evaluate(test_set)  # evaluate on test set only
    print(e)
    pass

In [40]:
try:
    joblib.dump(value=eval1,filename=f"outputs/{version}/test-evaluation.pkl")
    joblib.dump(value=evaluation_train,filename=f"outputs/{version}/train-evaluation.pkl")
    joblib.dump(value=evaluation_val,filename=f"outputs/{version}/val-evaluation.pkl")
except Expection as e:
    print(e)
    print('-----')
    print('saving successful test eval')
    joblib.dump(value=eval1,filename=f"outputs/{version}/test-evaluation.pkl")


# Run Live Prediction 

Load image as PIL

In [ ]:
from tensorflow.keras.preprocessing import image
import random 

pointer = int(random.randrange(len(test_set)))  # random test_set image 
label = labels[random.choice(labels)]  # random label selection

pil_image = image.load_img(test_path + '/' + label + '/' + os.listdir(test_path+'/' + label)[pointer],
                           target_size=image_shape, color_mode='rgb')
print(f'Image shape: {pil_image.size}, Image mode: {pil_image.mode}')
pil_image

Convert prediction image to array for prediction

In [ ]:
pred_img = image.img_to_array(pil_image)
pred_img = np.expand_dims(pred_img, axis=0)/255
print(pred_img.shape)
pred_img

Predict class probability on test image

In [ ]:
pred_proba = model.predict(pred_img)[0, 0]

target_map = {v: k for k, v in train_set.class_indices.items()}
pred_class = target_map[pred_proba > 0.5]  # define threshold between classes

if pred_class == target_map[0]:
  pred_proba = 1 - pred_proba

print(f'Prediction: {pred_class}\nConfidence: {pred_proba*100:.2f}%')

*** 

### Push all notebook outputs to repo

In [ ]:
try:
    !git add outputs/{version}
    !git commit -m f"Add " + {version} + " outputs for analysis"
    !git push origin main
except Exception as e:
    print(e)